In [0]:
# !pip install --upgrade tensorflow -q
# import os
# os._exit(0)

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from IPython.display import display
import time

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0'

In [0]:
# !unzip -q ml-100k.zip
# !rm ml-100k.zip

In [0]:
#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')

#Reading items file:
i_cols = ['movie_id', 'movie title' ,'release date','video release date', \
          'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', \
          'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', \
          'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

In [0]:
user, item, title = 'user_id', 'movie_id', 'movie title'

In [4]:
display(users); display(ratings); display(items);

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
rating_movie = ratings.merge(items[[item, title]])
rating_movie.head()

,user_id,movie_id,rating,unix_timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [6]:
id2movie, movie2id = {}, {}

n_users = users.shape[0]
n_items = items.shape[0]

ui_matrix = np.zeros((n_users, n_items), dtype=np.int8) # user-item matrix

for (_, ui, mi, rating, _, mt) in rating_movie.itertuples():
    id2movie[mi] = mt
    movie2id[mt] = mi
    ui_matrix[ui-1, mi-1] = rating

print(id2movie[1], movie2id['Toy Story (1995)'], sep='\n')

Toy Story (1995)
1


In [7]:
BUFFER_SIZE = len(rating_movie)
BATCH_SIZE = 128
STEPS_PER_EPOCH = len(rating_movie)//BATCH_SIZE

dataset = tf.data.Dataset.from_tensor_slices((rating_movie[[user, item]].values - 1, 
                                            rating_movie['rating'].values))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)

dataset

<BatchDataset shapes: ((None, 2), (None,)), types: (tf.int64, tf.int64)>

In [0]:
x, y = iter(dataset.take(STEPS_PER_EPOCH)).next()

### Matrix factorization

In [0]:
@tf.function
def train_step(user, item, rating, optimizer, loss_func):
    loss = 0

    with tf.GradientTape() as tape:
        output = model(user, item)
        loss += loss_func(rating, output)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss


def train(model,
          epochs=5, 
          optimizer=tf.keras.optimizers.Adam(),
          loss_func=tf.keras.losses.MeanSquaredError()):
    
    for epoch in range(epochs):
        start = time.time()
        total_loss = 0

        for (batch, (user_item, rating)) in enumerate(dataset.take(STEPS_PER_EPOCH)):
            user, item = zip(*user_item)
            user, item = np.array(user), np.array(item)

            batch_loss = train_step(user, item, rating, 
                                    optimizer, loss_func)
            total_loss += batch_loss

            if batch % 100 == 0:
                print('Epoch {} Batch {} Loss {:.2f}'.format(epoch + 1,
                                                        batch,
                                                        batch_loss.numpy()))
        print('Epoch {} Loss {:.2f}'.format(epoch + 1,
                                        total_loss / STEPS_PER_EPOCH))
        print('Time taken for 1 epoch {:2f} sec\n'.format(time.time() - start))

`users` and `items` has same embedding dimentions

In [0]:
class Collab(tf.keras.Model):
    def __init__(self, n_user, n_item, n_dim=50, y_range=(0, 10)):
        super().__init__()
        self.user_w = tf.keras.layers.Embedding(n_user, n_dim)
        self.item_w = tf.keras.layers.Embedding(n_item, n_dim)
        self.user_b = tf.keras.layers.Embedding(n_user, 1)
        self.item_b = tf.keras.layers.Embedding(n_item, 1)
        self.y_range = y_range
  
    def call(self, user, item):
        x = self.user_w(user) * self.item_w(item)
        x = tf.reduce_sum(x, axis=1)
        x = x + tf.squeeze(self.user_b(user) + self.item_b(item))
        x = tf.sigmoid(x) * (self.y_range[1] - self.y_range[0]) + self.y_range[0]

        return x

model = Collab(n_user=n_users, n_item=n_items,
               n_dim=30, y_range=(0, 5.5))

In [13]:
train(model)

Epoch 1 Batch 0 Loss 2.0119
Epoch 1 Batch 100 Loss 1.7786
Epoch 1 Batch 200 Loss 1.5770
Epoch 1 Batch 300 Loss 1.4461
Epoch 1 Batch 400 Loss 1.4388
Epoch 1 Batch 500 Loss 1.3316
Epoch 1 Batch 600 Loss 1.0915
Epoch 1 Batch 700 Loss 1.1325
Epoch 1 Loss 1.4379
Time taken for 1 epoch 21.36195969581604 sec

Epoch 2 Batch 0 Loss 0.9327
Epoch 2 Batch 100 Loss 0.9262
Epoch 2 Batch 200 Loss 1.0575
Epoch 2 Batch 300 Loss 0.7589
Epoch 2 Batch 400 Loss 0.7878
Epoch 2 Batch 500 Loss 0.9118
Epoch 2 Batch 600 Loss 0.9057
Epoch 2 Batch 700 Loss 0.9967
Epoch 2 Loss 0.9049
Time taken for 1 epoch 20.443655490875244 sec

Epoch 3 Batch 0 Loss 0.8703
Epoch 3 Batch 100 Loss 0.8120
Epoch 3 Batch 200 Loss 0.6941
Epoch 3 Batch 300 Loss 0.8047
Epoch 3 Batch 400 Loss 0.6483
Epoch 3 Batch 500 Loss 0.8333
Epoch 3 Batch 600 Loss 0.5834
Epoch 3 Batch 700 Loss 0.7663
Epoch 3 Loss 0.7901
Time taken for 1 epoch 20.484580516815186 sec

Epoch 4 Batch 0 Loss 0.5741
Epoch 4 Batch 100 Loss 0.6805
Epoch 4 Batch 200 Loss 0.799

`users` and `items` has different embedding dimentions

In [0]:
class Collab(tf.keras.Model):
    def __init__(self, n_user, n_item, user_emb_dim=50, item_emb_dim=50, y_range=(0, 10)):
        super().__init__()
        self.user_w = tf.keras.layers.Embedding(n_user, user_emb_dim, name='user_w')
        self.item_w = tf.keras.layers.Embedding(n_item, item_emb_dim, name='item_w')
        self.user_b = tf.keras.layers.Embedding(n_user, 1, name='user_b')
        self.item_b = tf.keras.layers.Embedding(n_item, 1, name='item_b')
        self.user_item_w = tf.Variable(tf.random.uniform(shape=(user_emb_dim, item_emb_dim)), name='user_item_w')
        self.y_range = y_range
  
    def call(self, user, item):
        x = tf.matmul(self.user_w(user), self.user_item_w)      # Shape: (batch_size x user_emb_dim) @ (user_emb_dim x item_emb_dim) ==> (batch_size x item_emb_dim)
        x = tf.reduce_sum(x * self.item_w(item), axis=1)        # Shape: (batch_size x item_emb_dim) x (batch_size x item_emb_dim) ==> (batch_size x item_emb_dim).sum() ==> (batch_size)

        x = x + tf.squeeze(self.user_b(user) + self.item_b(item))
        x = tf.sigmoid(x) * (self.y_range[1] - self.y_range[0]) + self.y_range[0]

        return x


model = Collab(n_user=n_users, n_item=n_items,
               user_emb_dim=33, item_emb_dim=35, 
               y_range=(0, 5.5))

In [11]:
train(model, epochs=10)

Epoch 1 Batch 0 Loss 1.7572
Epoch 1 Batch 100 Loss 1.9965
Epoch 1 Batch 200 Loss 1.2354
Epoch 1 Batch 300 Loss 1.2619
Epoch 1 Batch 400 Loss 1.1062
Epoch 1 Batch 500 Loss 0.9310
Epoch 1 Batch 600 Loss 0.8622
Epoch 1 Batch 700 Loss 1.1339
Epoch 1 Loss 1.1653
Time taken for 1 epoch 22.321846 sec

Epoch 2 Batch 0 Loss 0.9579
Epoch 2 Batch 100 Loss 0.9253
Epoch 2 Batch 200 Loss 0.8886
Epoch 2 Batch 300 Loss 0.8123
Epoch 2 Batch 400 Loss 0.8265
Epoch 2 Batch 500 Loss 0.9044
Epoch 2 Batch 600 Loss 0.6671
Epoch 2 Batch 700 Loss 0.7519
Epoch 2 Loss 0.9186
Time taken for 1 epoch 21.624687 sec

Epoch 3 Batch 0 Loss 0.9700
Epoch 3 Batch 100 Loss 0.8770
Epoch 3 Batch 200 Loss 0.7389
Epoch 3 Batch 300 Loss 0.8972
Epoch 3 Batch 400 Loss 0.8922
Epoch 3 Batch 500 Loss 0.9159
Epoch 3 Batch 600 Loss 1.0514
Epoch 3 Batch 700 Loss 0.9074
Epoch 3 Loss 0.8889
Time taken for 1 epoch 22.123711 sec

Epoch 4 Batch 0 Loss 0.9498
Epoch 4 Batch 100 Loss 0.6378
Epoch 4 Batch 200 Loss 0.7479
Epoch 4 Batch 300 Loss 1

In [12]:
model.summary()

Model: "collab"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  31119     
_________________________________________________________________
embedding_1 (Embedding)      multiple                  58870     
_________________________________________________________________
embedding_2 (Embedding)      multiple                  943       
_________________________________________________________________
embedding_3 (Embedding)      multiple                  1682      
Total params: 93,769
Trainable params: 93,769
Non-trainable params: 0
_________________________________________________________________
